In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/effecientnet-tf-h5/pytorch/default/1/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5
/kaggle/input/bangladeshi-local-flower/archive/Kamini/img_aug_370_hflip.jpg
/kaggle/input/bangladeshi-local-flower/archive/Kamini/img_aug_45_zoom.jpg
/kaggle/input/bangladeshi-local-flower/archive/Kamini/img_aug_407_vflip.jpg
/kaggle/input/bangladeshi-local-flower/archive/Kamini/img_aug_367_vflip.jpg
/kaggle/input/bangladeshi-local-flower/archive/Kamini/img_aug_3_vflip.jpg
/kaggle/input/bangladeshi-local-flower/archive/Kamini/img_origonly_535_orig.jpg
/kaggle/input/bangladeshi-local-flower/archive/Kamini/img_aug_245_hflip.jpg
/kaggle/input/bangladeshi-local-flower/archive/Kamini/img_aug_319_zoom.jpg
/kaggle/input/bangladeshi-local-flower/archive/Kamini/img_aug_310_zoom.jpg
/kaggle/input/bangladeshi-local-flower/archive/Kamini/img_aug_80_hflip.jpg
/kaggle/input/bangladeshi-local-flower/archive/Kamini/img_aug_372_orig.jpg
/kaggle/input/bangladeshi-local-flower/archive/Kami

In [2]:
!pip install tensorflow
!pip install numpy
!pip install pandas matplotlib scikit-learn

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os


2025-06-02 08:36:10.617764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748853370.817829      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748853370.876435      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os
from shutil import copyfile
import numpy as np

# Constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 123  # For reproducibility
data_dir = "/kaggle/input/bangladeshi-local-flower/archive"
output_dir = "/kaggle/working/split_data"  # Organized output directory

# Create output directories
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_dir, split), exist_ok=True)

# Get class names
class_names = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
print("Class names:", class_names)

# Split data into train (80%), val (10%), test (10%) with stratification
np.random.seed(SEED)  # Ensure reproducibility

for class_name in class_names:
    class_dir = os.path.join(data_dir, class_name)
    files = [f for f in os.listdir(class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    np.random.shuffle(files)  # Shuffle files to ensure random distribution
    
    # Split indices
    train_end = int(0.8 * len(files))
    val_end = train_end + int(0.1 * len(files))
    
    train_files = files[:train_end]
    val_files = files[train_end:val_end]
    test_files = files[val_end:]
    
    # Create class subdirectories in train/val/test
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(output_dir, split, class_name), exist_ok=True)
    
    # Copy files
    for f in train_files:
        src = os.path.join(class_dir, f)
        dst = os.path.join(output_dir, 'train', class_name, f)
        copyfile(src, dst)
    
    for f in val_files:
        src = os.path.join(class_dir, f)
        dst = os.path.join(output_dir, 'val', class_name, f)
        copyfile(src, dst)
    
    for f in test_files:
        src = os.path.join(class_dir, f)
        dst = os.path.join(output_dir, 'test', class_name, f)
        copyfile(src, dst)

# Load datasets with augmentation (only for training)
train_dataset = image_dataset_from_directory(
    os.path.join(output_dir, 'train'),
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    seed=SEED,
    label_mode='categorical',  # Better for multi-class classification
)

val_dataset = image_dataset_from_directory(
    os.path.join(output_dir, 'val'),
    shuffle=False,  # No need to shuffle validation data
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    label_mode='categorical',
)

test_dataset = image_dataset_from_directory(
    os.path.join(output_dir, 'test'),
    shuffle=False,  # No shuffling for test data
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    label_mode='categorical',
)

# Apply data augmentation to training set
augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

train_dataset = train_dataset.map(
    lambda x, y: (augmentation(x, training=True), y),
    num_parallel_calls=tf.data.AUTOTUNE,
)

# Optimize dataset loading
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

print(f"\nDataset sizes:")
print(f"- Training samples: {len(train_dataset) * BATCH_SIZE}")
print(f"- Validation samples: {len(val_dataset) * BATCH_SIZE}")
print(f"- Test samples: {len(test_dataset) * BATCH_SIZE}")

Class names: ['Belly', 'Gondhoraj', 'Kamini', 'Kathgulap', 'Kolaboti', 'Madhobilota', 'Noyontara', 'Rainlily', 'Sondhamaloti', 'Zinnia']
Found 16000 files belonging to 10 classes.


I0000 00:00:1748853551.572715      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1748853551.573393      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Found 2000 files belonging to 10 classes.
Found 2000 files belonging to 10 classes.

Dataset sizes:
- Training samples: 16000
- Validation samples: 2016
- Test samples: 2016


In [5]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)


In [6]:
import os
print(os.listdir('/kaggle/input'))  # List all datasets
print(os.listdir('/kaggle/input/bangladeshi-local-flower/archive'))  # List files in your dataset

['effecientnet-tf-h5', 'bangladeshi-local-flower', 'effecientnetv2']
['Kamini', 'Rainlily', 'Sondhamaloti', 'Kolaboti', 'Madhobilota', 'Gondhoraj', 'Kathgulap', 'Noyontara', 'Zinnia', 'Belly']


In [7]:
import os

# Check what's in the potential weight folders
print(os.listdir('/kaggle/input/effecientnet-tf-h5/pytorch/default/1'))
print(os.listdir('/kaggle/input/effecientnetv2/pytorch/default/1'))

['mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5']
['best_model_efficientnetv2.pth']


In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os

# ====== CONFIGURATION ======
DATASET_PATH = '/kaggle/input/bangladeshi-local-flower/archive'
MODEL_WEIGHTS_PATH = '/kaggle/input/efficientnet-tf-h5/efficientnetv2-b0_notop.h5'  # Update this if path is different
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
CLASS_NAMES = sorted(os.listdir(DATASET_PATH))  # Automatically get class names

# ====== DATA LOADING ======
def load_datasets():
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        DATASET_PATH,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE)
    
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        DATASET_PATH,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE)
    
    return train_ds, val_ds

# ====== MODEL LOADING WITH ERROR HANDLING ======
def load_model_with_fallback():
    # First try loading from local path if it exists
    if os.path.exists(MODEL_WEIGHTS_PATH):
        try:
            model = tf.keras.applications.EfficientNetV2B0(
                weights=None,  # We'll load weights manually
                include_top=False,
                input_shape=IMG_SIZE + (3,)
            )
            model.load_weights(MODEL_WEIGHTS_PATH)
            print("✅ Successfully loaded weights from local path")
            return model
        except Exception as e:
            print(f"⚠️ Couldn't load custom weights from {MODEL_WEIGHTS_PATH}: {e}")
    
    # Fallback options
    fallback_options = [
        'imagenet',  # Try to download
        None  # Use random initialization if all else fails
    ]
    
    for option in fallback_options:
        try:
            print(f"🔄 Trying fallback option: {option}")
            model = tf.keras.applications.EfficientNetV2B0(
                weights=option,
                include_top=False,
                input_shape=IMG_SIZE + (3,)
            )
            print(f"✅ Successfully loaded with option: {option}")
            return model
        except Exception as e:
            print(f"⚠️ Failed to load with option {option}: {e}")
    
    raise RuntimeError("All model loading options failed")

# ====== MAIN EXECUTION ======
if __name__ == "__main__":
    # 1. Load data
    train_dataset, val_dataset = load_datasets()
    print(f"📊 Found {len(CLASS_NAMES)} classes: {CLASS_NAMES}")
    
    # 2. Build model
    base_model = load_model_with_fallback()
    base_model.trainable = False
    
    model = tf.keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(len(CLASS_NAMES), activation='softmax')
    ])
    
    # 3. Compile and train
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.summary()
    
    # 4. Verify everything works
    print("🧪 Running test prediction...")
    test_image = tf.random.normal((1, *IMG_SIZE, 3))
    prediction = model.predict(test_image)
    print(f"🎯 Test prediction shape: {prediction.shape}")
    print("🚀 Model is ready for training!")

Found 20000 files belonging to 10 classes.
Using 16000 files for training.
Found 20000 files belonging to 10 classes.
Using 4000 files for validation.
📊 Found 10 classes: ['Belly', 'Gondhoraj', 'Kamini', 'Kathgulap', 'Kolaboti', 'Madhobilota', 'Noyontara', 'Rainlily', 'Sondhamaloti', 'Zinnia']
🔄 Trying fallback option: imagenet
24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
✅ Successfully loaded with option: imagenet


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetv2-b0 (Functional)       │ (None, 7, 7, 1280)          │       5,919,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 10)                  │          12,810 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,932,122 (22.63 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 5,919,312 (22.58 MB)

🧪 Running test prediction...


I0000 00:00:1748853586.821793     120 service.cc:148] XLA service 0x79778c056590 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748853586.822694     120 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1748853586.822712     120 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1748853587.703051     120 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
🎯 Test prediction shape: (1, 10)
🚀 Model is ready for training!


I0000 00:00:1748853591.774442     120 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [9]:
from sklearn.utils import class_weight
import numpy as np

# Calculate class weights
train_labels = np.concatenate([y for x, y in train_dataset], axis=0)
class_weights = class_weight.compute_class_weight('balanced', 
                                                classes=np.unique(train_labels), 
                                                y=train_labels)
class_weights = dict(enumerate(class_weights))

# Use in model.fit:
# model.fit(..., class_weight=class_weights)

In [10]:
train_dataset = train_dataset.map(
    lambda x, y: (augmentation(x, training=True), y),
    num_parallel_calls=tf.data.AUTOTUNE
).cache().prefetch(tf.data.AUTOTUNE)  # Caching for speed


In [11]:
base_model = tf.keras.applications.EfficientNetV2M(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)  # Added missing parenthesis
)

214201816/214201816 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [12]:
import tensorflow as tf
from tensorflow.keras import layers

# Correct model definition
base_model = tf.keras.applications.EfficientNetV2M(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)  # Now properly closed
)

# Build your model
model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])

# Freeze base model initially
base_model.trainable = False

# Compile
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
base_model.trainable = True
for layer in base_model.layers[:-10]:  # Unfreeze last 10 layers
    layer.trainable = False

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

In [15]:
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))

In [16]:
# Phase 1: Frozen base
history = model.fit(train_dataset, epochs=7, validation_data=val_dataset)

# Phase 2: Unfreeze some layers
base_model.trainable = True
for layer in base_model.layers[:-10]:
    layer.trainable = False
    
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Lower LR
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_dataset, epochs=5, validation_data=val_dataset)

Epoch 1/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 214s 249ms/step - accuracy: 0.8736 - loss: 0.3968 - val_accuracy: 0.9910 - val_loss: 0.0304
Epoch 2/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 101s 203ms/step - accuracy: 0.9789 - loss: 0.0641 - val_accuracy: 0.9950 - val_loss: 0.0111
Epoch 3/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 100s 201ms/step - accuracy: 0.9850 - loss: 0.0460 - val_accuracy: 0.9958 - val_loss: 0.0150
Epoch 4/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 101s 201ms/step - accuracy: 0.9857 - loss: 0.0484 - val_accuracy: 0.9962 - val_loss: 0.0121
Epoch 5/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 101s 202ms/step - accuracy: 0.9892 - loss: 0.0361 - val_accuracy: 0.9937 - val_loss: 0.0268
Epoch 6/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 100s 201ms/step - accuracy: 0.9913 - loss: 0.0274 - val_accuracy: 0.9942 - val_loss: 0.0215
Epoch 7/7
500/500 ━━━━━━━━━━━━━━━━━━━━ 101s 201ms/step - accuracy: 0.9928 - loss: 0.0218 - val_accuracy: 0.9965 - val_loss: 0.0090
Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 235ms/step - accuracy: 0.9926 - loss: 0

In [18]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [19]:
 # Evaluate on test dataset
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}")


63/63 ━━━━━━━━━━━━━━━━━━━━ 46s 298ms/step - accuracy: 0.9992 - loss: 0.0029
Test Accuracy: 0.9990


In [22]:
y_true = []
y_pred = []

for images, labels in test_dataset:
    preds = model.predict(images)
    y_true.extend(np.argmax(labels.numpy(), axis=1))
    y_pred.extend(np.argmax(preds, axis=1))

print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion matrix
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 